# Spatial Oregonator (2D) — operator splitting with macrofor + SymPy

This notebook demonstrates a complete pipeline:
- Model: the (dimensionless) Oregonator reaction model on a 2D grid with diffusion.
- Time stepping: operator splitting — explicit diffusion, implicit reaction (per-grid-point backward Euler solved with Newton).
- The Jacobian and residual for the Newton solver are generated symbolically with SymPy and inserted into Fortran code using the `macrofor` helpers. The full Fortran program is written with `genfor()`.
- The explicit diffusion timestep is chosen automatically by a CFL-like stability criterion.
- The Fortran program writes PGM images each output step; the notebook reads the PGM files and composes an animated GIF.

Requirements: sympy, macrofor (this repo), numpy, pillow, matplotlib (optional), and a Fortran compiler (`gfortran`).

In [ ]:
# Imports
import sympy as sp
from sympy import I
from sympy import fcode
from textwrap import dedent
from macrofor import api as mf
from pathlib import Path
import numpy as np
from PIL import Image
import shutil
import subprocess
print('SymPy', sp.__version__)

## Model and discretization
We use a 2-variable Oregonator form (dimensionless):
du/dt = (1/eps) * (u - u**2 - f*v*(u - q)/(u + q)) + D_u * Laplacian(u)
dv/dt = u - v + D_v * Laplacian(v)

Operator splitting: for each timestep dt:
1. Explicit diffusion update (2D five-point Laplacian, forward Euler)
2. Implicit reaction update per grid point using backward Euler solved with Newton:
   solve U_new = U_mid + dt * R(U_new)  (R is reaction vector)

We compute dt from explicit diffusion stability: dt <= dx^2 /(4*D_max) and add a safety factor.

In [ ]:
# Parameters
nx, ny = 128, 128   # grid
Lx, Ly = 10.0, 10.0
dx = Lx / (nx - 1)
dy = Ly / (ny - 1)

# Oregonator parameters (example values)
eps = 0.02
f_par = 1.4
q = 0.0002

D_u = 0.01
D_v = 0.005

# time control
t_final = 10.0
output_interval = 0.1   # seconds between PGM outputs (physical time)

# stability dt for explicit diffusion (2D): dt <= dx^2 / (4*D_max)
D_max = max(D_u, D_v)
dt_diff = 0.5 * (dx*dx) / (4.0 * D_max)   # safety factor 0.5
print('dx,dy =', dx, dy)
print('dt (diffusion stability) =', dt_diff)

## Symbolic reaction terms and Jacobian (SymPy)
We build reaction vector R = [R_u(u,v), R_v(u,v)] and compute residual for backward Euler and Jacobian dR/dU.

In [ ]:
# Symbolic variables
u_s, v_s, dt_s, eps_s, f_s, q_s = sp.symbols('u v dt eps f q')

# Reaction terms R(u,v) (no diffusion)
R_u = (1/eps_s) * (u_s - u_s**2 - f_s * v_s * (u_s - q_s) / (u_s + q_s))
R_v = u_s - v_s

# Backward Euler residual for implicit solve: res(U_new) = U_new - U_old - dt * R(U_new)
u_new, v_new, u_old, v_old = sp.symbols('u_new v_new u_old v_old')
res_u = u_new - u_old - dt_s * R_u.subs({u_s: u_new, v_s: v_new})
res_v = v_new - v_old - dt_s * R_v.subs({u_s: u_new, v_s: v_new})

# Jacobian of the residual wrt [u_new, v_new]
J11 = sp.simplify(sp.diff(res_u, u_new))
J12 = sp.simplify(sp.diff(res_u, v_new))
J21 = sp.simplify(sp.diff(res_v, u_new))
J22 = sp.simplify(sp.diff(res_v, v_new))

print('Residuals and Jacobian symbolic expressions prepared')

## Convert SymPy expressions to Fortran RHS strings
We will generate a small Fortran fragment that computes residuals and Jacobian entries for a given node; macrofor will assemble the full program.

In [ ]:
def fortran_rhs_from_expr(expr, assign_to='TMP'):
    s = fcode(expr, assign_to=assign_to, source_format='free')
    rhs = s.split('=', 1)[1].strip()
    return rhs

# produce Fortran RHS snippets that use u_new,v_new,u_old,v_old,dt,eps,f,q
subs_map = {dt_s: sp.symbols('dt'), eps_s: sp.symbols('eps'), f_s: sp.symbols('f'), q_s: sp.symbols('q')}

res_u_rhs = fortran_rhs_from_expr(res_u.subs({dt_s: sp.symbols('dt'), eps_s: sp.symbols('eps'), f_s: sp.symbols('f'), q_s: sp.symbols('q')}), assign_to='RES_U')
res_v_rhs = fortran_rhs_from_expr(res_v.subs({dt_s: sp.symbols('dt')}), assign_to='RES_V')
J11_rhs = fortran_rhs_from_expr(J11.subs({dt_s: sp.symbols('dt'), eps_s: sp.symbols('eps'), f_s: sp.symbols('f'), q_s: sp.symbols('q')}), assign_to='J11')
J12_rhs = fortran_rhs_from_expr(J12.subs({dt_s: sp.symbols('dt'), eps_s: sp.symbols('eps'), f_s: sp.symbols('f'), q_s: sp.symbols('q')}), assign_to='J12')
J21_rhs = fortran_rhs_from_expr(J21.subs({dt_s: sp.symbols('dt')}), assign_to='J21')
J22_rhs = fortran_rhs_from_expr(J22.subs({dt_s: sp.symbols('dt')}), assign_to='J22')

print('Generated Fortran RHS snippets (samples):')
print('res_u:', res_u_rhs[:200])
print('J11:', J11_rhs[:200])

## Build Fortran program with macrofor
Program outline (high-level):
- Declarations and initialization
- time loop: for each dt step:
  - explicit diffusion update for u,v (in-place or to temporary arrays)
  - for each grid point: implicit reaction solve via Newton (small 2x2 linear solve)
  - write output PGM every output_interval

We assemble the program with `macrofor.api.programm` and friends and write to file with `genfor()`.

In [ ]:
def generate_fortran_oregonator(output_path: Path):
    from macrofor.api import genfor, programm, declarem, equalf, dom, openm
    
    # Helper: declarations
    decls = [
        declarem('integer', ['nx', 'ny', 'i', 'j', 'step', 'maxsteps']),
        declarem('double precision', ['dx', 'dy', 'dt', 't', 'output_interval', 'eps', 'f_par', 'q_par']),
        declarem('double precision', ['D_u', 'D_v']),
        # flattened arrays (column-major) for simplicity: u(1:nx,1:ny) => u(nx,ny)
        'double precision, allocatable :: u(:,:), v(:,:), u_tmp(:,:), v_tmp(:,:)',
        declarem('integer', ['out_step']),
    ]
    
    inits = [
        equalf('nx', str({nx})),",
        equalf('ny', str({ny})),",
        equalf('dx', f'{dx}d0'),",
        equalf('dy', f'{dy}d0'),",
        equalf('eps', str(eps)),",
        equalf('f_par', str(f_par)),",
        equalf('q_par', str(q)),",
        equalf('D_u', str(D_u)),",
        equalf('D_v', str(D_v)),",
        equalf('dt', '0.0d0'),",
        equalf('t', '0.0d0'),",
        equalf('output_interval', f'{output_interval}d0'),",
        equalf('out_step', '0'),",
        "allocate(u(nx,ny), v(nx,ny), u_tmp(nx,ny), v_tmp(nx,ny))",
        "u = 0.5d0 + 0.01d0 * randn(nx,ny)  ! initial condition (placeholder, randn to be replaced)",
        "v = 0.5d0 + 0.01d0 * randn(nx,ny)",
    ]
    
    # Fortran fragment: diffusion explicit (5-point Laplacian)
    diff_body = [
        "do j = 2, ny-1",
        "  do i = 2, nx-1",
    '  u_tmp(i,j) = u(i,j) + dt * D_u * ((u(i+1,j)-2.0d0*u(i,j)+u(i-1,j))/dx**2 + (u(i,j+1)-2.0d0*u(i,j)+u(i,j-1))/dy**2)',",
    '  v_tmp(i,j) = v(i,j) + dt * D_v * ((v(i+1,j)-2.0d0*v(i,j)+v(i-1,j))/dx**2 + (v(i,j+1)-2.0d0*v(i,j)+v(i,j-1))/dy**2)',",
        "  end do",
        "end do",
        "u = u_tmp",
        "v = v_tmp",
    ]
    
    # Fortran fragment: implicit reaction solved by Newton (per node)
    # We'll insert the symbolically generated residual/Jacobian code as text lines here.
    reaction_frag = []
    # Newton loop per node (pseudocode assembled as lines):
    reaction_frag += [
        "do j = 1, ny",
        "  do i = 1, nx",
        "    u_old = u(i,j)",
        "    v_old = v(i,j)",
        "    u_new = u_old",
        "    v_new = v_old",
        "    do 10 iter = 1, 20",
        "      ! compute residuals RES_U, RES_V and Jacobian J11,J12,J21,J22",
        '      RES_U = {res_u_rhs}',",
        '      RES_V = {res_v_rhs}',",
        '      J11 = {J11_rhs}',",
        '      J12 = {J12_rhs}',",
        '      J21 = {J21_rhs}',",
        '      J22 = {J22_rhs}',",
        "      ! solve 2x2 linear system for delta (use Cramer's rule)",
        "      detJ = J11*J22 - J12*J21",
        "      if (abs(detJ) .lt. 1.0d-16) detJ = sign(1.0d-16, detJ)",
        "      du = -( J22*RES_U - J12*RES_V ) / detJ",
        "      dv = -(-J21*RES_U + J11*RES_V ) / detJ",
        "      u_new = u_new + du",
        "      v_new = v_new + dv",
        "      if (abs(du) .lt. 1.0d-10 .and. abs(dv) .lt. 1.0d-10) exit",
        "    end do",
        "    u(i,j) = u_new",
        "    v(i,j) = v_new",
        "  end do",
        "end do",
    ]
    
    # Output block (write PGM) - we'll use simple integer scaling
    write_block = [
        "open(unit=10, file='frame_'//trim(adjustl(str(out_step)))//'.pgm', status='replace')",
        "write(10,*) 'P2'",
        "write(10,*) nx, ny",
        'write(10,*) 255',",
        "do j = 1, ny",
        "  do i = 1, nx",
    '  write(10,'\"(I3,1X)\"', advance=\"no\") int(255.0d0 * min(max(u(i,j),0.0d0),1.0d0))',",
  "  end do",
  "  write(10,*)",
  "end do",
  "close(10)",
    ]
    
    prog_body = inits + [''] + diff_body + [''] + reaction_frag + [''] + ["out_step = out_step + 1"] + write_block
    prog = programm('oregonator_spatial', decls + prog_body)
    
    genfor(output_path, [prog])
    print('Wrote Fortran to', output_path)

output_f90 = Path('oregonator_spatial.f90')
generate_fortran_oregonator(output_f90)

## Compile & run (example)
Use a Fortran compiler to build and run the program. Example (PowerShell):
```powershell
gfortran -O2 oregonator_spatial.f90 -o oregonator_spatial.exe
.\oregonator_spatial.exe
```
The program writes frames named `frame_*.pgm`.

## Postprocessing: read PGMs and make GIF

In [ ]:
# Read generated PGM frames and compose an animated GIF
def frames_to_gif(pattern='frame_*.pgm', out='oregonator.gif'):
    import glob
    files = sorted(glob.glob(pattern))
    if not files:
        print('No frames found (run the Fortran program first)')
        return
    imgs = [Image.open(f).convert('P') for f in files]
    imgs[0].save(out, save_all=True, append_images=imgs[1:], duration=100, loop=0)
    print('Wrote GIF', out)

# Example (uncomment after running Fortran):
# frames_to_gif()